In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from IPython.display import clear_output

In [3]:
! git clone https://github.com/jeromerony/fast_adversarial.git

Cloning into 'fast_adversarial'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 84 (delta 0), reused 0 (delta 0), pack-reused 81
Unpacking objects: 100% (84/84), 240.47 KiB | 2.43 MiB/s, done.


In [4]:
!pip install git+https://github.com/jeromerony/fast_adversarial
clear_output()

In [5]:
! git clone https://github.com/hojat-ardi/image-project.git

Cloning into 'image-project'...
remote: Enumerating objects: 308, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 308 (delta 63), reused 147 (delta 43), pack-reused 111
Receiving objects: 100% (308/308), 39.82 MiB | 30.43 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [6]:
%cd /content
!git clone https://github.com/alirezaabdollahpour/ML_Graduate_Project_TA.git

/content
Cloning into 'ML_Graduate_Project_TA'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 40 (delta 10), reused 29 (delta 7), pack-reused 0
Unpacking objects: 100% (40/40), 7.41 MiB | 12.40 MiB/s, done.


In [25]:
#copy model
!cp  /content/image-project/ImageProject-master/phase3/Eliminating_of_BatchNorm/resnet_Eliminating_of_BatchNorm.py  /content/fast_adversarial/examples

In [8]:
#copy model
!cp /content/ML_Graduate_Project_TA/Phase_1/utils.py  /content/fast_adversarial/examples

In [26]:
%%shell
cat > /content/fast_adversarial/examples/Cifar10_example.py << EOF


import argparse
import torch
import time
from torch.utils import data
from torchvision import datasets, transforms
from torchvision.utils import save_image
from resnet_Eliminating_of_BatchNorm import*
from utils import*
# from fast_adv.models.mnist import SmallCNN
from fast_adv.attacks import DDN, CarliniWagnerL2
from fast_adv.utils import requires_grad_, l2_norm


if __name__ == '__main__':
    parser = argparse.ArgumentParser('Generate adversarial examples on CFAR-10')
    parser.add_argument('--data-path', default='data/cifar10')

    args = parser.parse_args()

    torch.manual_seed(42)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    print('Loading data')
    dataset = datasets.CIFAR10(args.data_path, train=False,
                             transform=transforms.ToTensor(),
                             download=True)
    loader = data.DataLoader(dataset, shuffle=False, batch_size=1000)

    x, y = next(iter(loader))
    x = x.to(device)
    y = y.to(device)
    # Inverse normalization
    mean = torch.tensor([0.4914, 0.4822, 0.4465]).view(1, 3, 1, 1)
    std = torch.tensor([0.2023, 0.1994, 0.2010]).view(1, 3, 1, 1)

    # Move std and mean tensors to the same device as x
    std = std.to(x.device)
    mean = mean.to(x.device)

    print('Loading model')
    model = ResNet18()
    model = nn.DataParallel(model)
    checkpoint = torch.load(r"/content/drive/MyDrive/weight/Eliminating_of_BatchNorm_ResNet18.pth")
    model.load_state_dict(checkpoint['net'])
    model = NormalizedModel(model, mean =  mean , std = std)
    model.eval().to(device)
    requires_grad_(model, False)


    y_pred = model(x)
    _, predicted = y_pred.max(1)
    total = y.size(0)
    correct = predicted.eq(y).sum().item()
    print("Clean accuracy is: " , 100.*correct/total, '%')


    print('Running DDN attack')
    attacker = DDN(steps=1000, device=device)
    start = time.time()
    ddn_atk = attacker.attack(model, x, labels=y, targeted=False)
    ddn_time = time.time() - start

    # print('Running C&W attack')
    # cwattacker = CarliniWagnerL2(device=device,
    #                              image_constraints=(0, 1),
    #                              num_classes=10)

    # start = time.time()
    # cw_atk = cwattacker.attack(model, x, labels=y, targeted=False)
    # cw_time = time.time() - start

    # Save images
    all_imgs = torch.cat((x, ddn_atk))
    save_image(all_imgs, 'images_and_attacks.png', nrow=16, pad_value=0)

    # Print metrics
    # pred_orig = model(x).argmax(dim=1).cpu()
    # pred_cw = model(cw_atk).argmax(dim=1).cpu()
    pred_ddn = model(ddn_atk).argmax(dim=1).cpu()
    # print('Predictions on original images: {}'.format(pred_orig))
    # print('Predictions on C&W attack: {}'.format(pred_cw))
    print('Predictions on DDN attack: {}'.format(pred_ddn))
    # print('C&W done in {:.1f}s: Success: {:.2f}%, Mean L2: {:.4f}.'.format(
    #     cw_time,
    #     (pred_cw != y.cpu()).float().mean().item() * 100,
    #     l2_norm(cw_atk - x).mean().item()
    # ))
    print('DDN 1000 done in {:.1f}s   \nSuccess: {:.2f}%   \nMean L2: {:.4f}   \nMedian L2: {:.4f}'.format(
        ddn_time,
        (pred_ddn != y.cpu()).float().mean().item() * 100,
        l2_norm(ddn_atk - x).mean().item(),
        l2_norm(ddn_atk - x).median().item()
    ))


/bin/bash: line 95: warning: here-document at line 0 delimited by end-of-file (wanted `EOF')


In [27]:
! python /content/fast_adversarial/examples/Cifar10_example.py

Loading data
Files already downloaded and verified
Loading model
Clean accuracy is:  93.4 %
Running DDN attack
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Predictions on DDN attack: tensor([5, 0, 1, 9, 2, 3, 9, 4, 5, 9, 5, 1, 3, 5, 8, 6, 7, 4, 0, 2, 0, 2, 2, 1,
        4, 0, 3, 6, 1, 2, 3, 3, 8, 3, 0, 8, 7, 9, 1, 3, 5, 2, 7, 2, 2, 1, 2, 8,
        5, 4, 6, 9, 2, 5, 0, 9, 4, 3, 5, 3, 5, 5, 1, 1, 3, 6, 8, 6, 5, 4, 5, 3,
        0, 0, 1, 7, 0, 5, 5, 0, 0, 0, 9, 5, 7, 7, 7, 3, 9, 1, 1, 6, 9, 2, 